In [ ]:
# ============ IMPORTS ==============
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
# ============ CONFIGURACIÓN ==============
IMG_SIZE = (48, 48)
BATCH_SIZE = 32
EPOCHS = 10
NUM_CLASSES = 7  # angry, disgust, fear, happy, sad, surprise, neutral

In [ ]:
# ============ PREPARACIÓN DE DATOS ==============
def create_data_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        validation_split=0.2
    )

    train_generator = train_datagen.flow_from_directory(
        'train_dir',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        color_mode='grayscale',
        class_mode='categorical',
        subset='training'
    )

    val_generator = train_datagen.flow_from_directory(
        'train_dir',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        color_mode='grayscale',
        class_mode='categorical',
        subset='validation'
    )

    return train_generator, val_generator

In [ ]:
# ============ MODELO DESDE CERO ==============
def create_cnn_model():
    model = keras.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),
        
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),
        
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),
        
        layers.Conv2D(256, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),
        
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    return model

In [ ]:
# ============ TRANSFER LEARNING (VGG16) ==============
def create_transfer_model():
    base_model = VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=(48,48,3)
    )
    
    # Congelar capas base
    base_model.trainable = False
    
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    return model

In [ ]:
# ============ FINE-TUNING ==============
def create_fine_tuned_model():
    base_model = VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=(48,48,3)
    )
    
    # Descongelar últimas capas
    base_model.trainable = True
    for layer in base_model.layers[:-4]:
        layer.trainable = False
    
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    return model

In [ ]:
# ============ ENTRENAMIENTO ==============
def train_model(model, train_gen, val_gen, model_name):
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    callbacks = [
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7),
        keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
    ]
    
    history = model.fit(
        train_gen,
        epochs=EPOCHS,
        validation_data=val_gen,
        callbacks=callbacks
    )
    
    model.save(f'{model_name}_emotion.h5')
    return history

In [ ]:
# ============ IMPLEMENTACIÓN EN TIEMPO REAL ==============
class EmotionDetector:
    def __init__(self, model_path):
        self.model = keras.models.load_model(model_path)
        self.emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    def detect_emotion(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)
        
        for (x, y, w, h) in faces:
            face_roi = gray[y:y+h, x:x+w]
            face_roi = cv2.resize(face_roi, IMG_SIZE)
            face_roi = face_roi.astype('float32') / 255.0
            face_roi = np.expand_dims(face_roi, axis=(0, -1))
            
            # Para modelos que esperan entrada RGB
            if self.model.input_shape[-1] == 3:
                face_roi = np.repeat(face_roi, 3, axis=-1)
            
            prediction = self.model.predict(face_roi, verbose=0)
            emotion_idx = np.argmax(prediction)
            confidence = np.max(prediction)
            
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            label = f"{self.emotions[emotion_idx]} ({confidence:.2f})"
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        
        return frame
    
    def run_camera(self):
        cap = cv2.VideoCapture(0)
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            frame = self.detect_emotion(frame)
            cv2.imshow('Emotion Detection', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
# ============ EJECUCIÓN PRINCIPAL ==============
if __name__ == "__main__":
    # Preparar datos
    train_gen, val_gen = create_data_generators()
    
    # Entrenar diferentes modelos
    models = {
        'cnn': create_cnn_model(),
        'transfer': create_transfer_model(),
        'fine_tuned': create_fine_tuned_model()
    }
    
    # Comparar modelos (ejecutar solo uno para ahorrar tiempo)
    best_model = None
    best_val_acc = 0
    
    for name, model in models.items():
        print(f"\nEntrenando {name}...")
        history = train_model(model, train_gen, val_gen, name)
        
        val_acc = max(history.history['val_accuracy'])
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = name
    
    print(f"\nMejor modelo: {best_model} con val_accuracy: {best_val_acc:.4f}")
    
    # Usar el mejor modelo para la cámara
    detector = EmotionDetector(f'{best_model}_emotion.h5')
    detector.run_camera()